In [3]:
import pandas as pd
import os
import numpy as np
import logging
import sys
import torch
import copy
import yaml
import random

from prediction_utils.pytorch_utils.metrics import (
    StandardEvaluator,
    FairOVAEvaluator,
    CalibrationEvaluator
)

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style("ticks")


args = {'cohort_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv',
        'base_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts',
        'grp_label_dict': {1: 'Black women', 2: 'White women', 3: 'Black men', 4: 'White men', 'overall': 'Overall'},
        'plot_path': os.path.join('/labs/shahlab/projects/agataf/fairness_utility/eval_manuscript/plots', 'paper_plots'),

       }

output_path = os.path.join(args['plot_path'], 'risk_category_counts.png')
os.makedirs(args['plot_path'], exist_ok=True)


In [1]:
def get_frac_treated(df, add_strata=None):
    base_grouping = ['model_type', 'model_id','labels'] 
    if add_strata:
        base_grouping.append(add_strata)
    num_treated_by_stratum = (df
         .query('(phase=="test")')
         .filter(['person_id', 'treat'] + base_grouping)
         .groupby(['treat'] + base_grouping)
         .count()
        )

    num_per_stratum = (df
          .query('(phase=="test")')
               .filter(['person_id'] + base_grouping)
               .groupby(base_grouping)
               .count()
         )

    frac_treated = num_treated_by_stratum.div(num_per_stratum)


    return frac_treated.reset_index()

In [10]:
preds

,phase,pred_probs,labels,weights,group,model_type,person_id,treat,relative_risk
0,9,0.005016,0,1.069459,2,original_pce,0,0,1.000000
1,1,0.015671,0,1.069459,2,original_pce,1,0,1.000000
2,2,0.016608,0,1.069459,2,original_pce,2,0,1.000000
3,test,0.017007,0,1.069459,2,original_pce,3,0,1.000000
4,5,0.123553,0,1.069459,2,original_pce,4,1,0.771363
...,...,...,...,...,...,...,...,...,...
25614,3,0.178557,0,1.000650,3,original_pce,25614,1,0.794772
25615,2,0.071152,0,1.090399,3,original_pce,25615,0,1.000000
25616,2,0.099609,0,1.096160,3,original_pce,25616,1,0.778980
25617,eval,0.280217,0,1.030498,3,original_pce,25617,2,0.576898


In [23]:
frac_treated_all = []
eqodds_threshold = [0.1, 1]
for experiment in ['original_pce', 'revised_pce', 'apr14_erm', 'apr14_erm_recalib', 'apr14_thr']:

    aggregate_path = os.path.join(args['base_path'], 'experiments', 
                                      experiment, 'performance',
                                      'all')
    preds_path = os.path.join(aggregate_path, 'predictions.csv')
    preds = pd.read_csv(preds_path)
    
    if experiment in ['apr14_mmd', 'apr14_thr']:
        preds = preds.query('model_id == @eqodds_threshold')
    if experiment not in ['original_pce', 'revised_pce']:
        preds = preds.query('fold_id == 1')
        
    if 'model_id' not in preds.columns:
        preds = preds.assign(model_id=0)
        
    frac_treated = preds.query("phase=='test'").filter(['labels', 'person_id', 
                                                        'model_id', 'model_type', 'treat',
                                                       'group'])
    frac_treated_all.append(frac_treated)

frac_treated_all = (pd
                    .concat(frac_treated_all)
                    .assign(model_type = lambda x: x.model_type.map({'original_pce': 'PCE',
                                                                     'revised_pce': 'rPCE',
                                                                     'erm': 'BL',
                                                                     'recalib_erm': 'rBL',
                                                                     'eqodds_thr': 'EO',
                                                                    }
                                                                   )
                           )
                   )



model_type = np.where((frac_treated_all.model_type=='EO') & (frac_treated_all.model_id==0.1), 'mEO', frac_treated_all.model_type)
model_type = np.where((frac_treated_all.model_type=='EO') & (frac_treated_all.model_id==1.0), 'sEO', model_type)


frac_treated_all = frac_treated_all.assign(model_type=model_type)


In [70]:
a=(frac_treated_all
 .filter(['person_id', 'model_type', 'treat'])
 .pivot(index='person_id', columns = 'model_type', values='treat')
 .dropna()
 .rename_axis(None, axis=1)
)

In [80]:
a

,BL,PCE,mEO,rBL,rPCE,sEO
person_id,,,,,,
3,0.0,0.0,0.0,0.0,0.0,1.0
21,0.0,0.0,0.0,0.0,0.0,1.0
42,0.0,0.0,0.0,0.0,0.0,0.0
43,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
23446,0.0,1.0,0.0,0.0,0.0,1.0
23457,2.0,2.0,2.0,2.0,2.0,1.0
23460,1.0,1.0,1.0,1.0,1.0,1.0


In [74]:
(a.min(axis=1)-a.max(axis=1)).abs().max()


2.0

In [79]:
a[((a.min(axis=1)-a.max(axis=1)).abs() ==2)].filter(['PCE', 'rPCE', 'BL', 'rBL', 'mEO', 'sEO'])

,PCE,rPCE,BL,rBL,mEO,sEO
person_id,,,,,,
4920,2.0,0.0,1.0,1.0,1.0,1.0
13600,1.0,0.0,2.0,2.0,2.0,1.0
18228,2.0,1.0,0.0,0.0,0.0,1.0
21031,2.0,1.0,0.0,0.0,1.0,1.0
22050,0.0,0.0,2.0,1.0,2.0,1.0
23268,2.0,1.0,1.0,0.0,1.0,1.0
